
Italy Energy System Optimization - Brownfield Analysis with ETS1 and ETS2
==========================================================================

This script creates an energy system optimization model for Italy as a whole system,
studying the effects of ETS1 (EU Emissions Trading System) and ETS2 (new ETS for 
buildings and transport) on the optimal design and operation of energy technologies.

The analysis includes:
1. Baseline optimization without ETS
2. Optimization with ETS1 only
3. Optimization with both ETS1 and ETS2
4. Comparative analysis of results

In [1]:
pip install adopt_net0

Note: you may need to restart the kernel to use updated packages.


In [2]:
import adopt_net0 as adopt
import json
from pathlib import Path
import os
import pandas as pd
import numpy as np

# Create folders 
results_data_path = Path("./userData")
results_data_path.mkdir(parents=True, exist_ok=True)

# create input data path and optimization templates 
input_data_path = Path("./macro_decarbonisation")
input_data_path.mkdir(parents=True, exist_ok=True)

# Create template input JSONs
adopt.create_optimization_templates(input_data_path)

Files already exist: macro_decarbonisation\Topology.json macro_decarbonisation\ConfigModel.json


In [3]:
results_data_path = Path("./results_macroperspective")
scenario_name = "baseline_no_ETS"

In [4]:
# Load json template
with open(input_data_path / "Topology.json", "r") as json_file:
    topology = json.load(json_file)
# Nodes
topology["nodes"] = ["northwest", "northeast", "center", "south", "islands"]
# Carriers: The Carries/ Vectors we have in the CGE model are gas, oil, electricity (Should I keep Hydrogen and also add CCS as before)
topology["carriers"] = ["electricity", "heat", "gas", "hydrogen"]  
# Investment periods:
topology["investment_periods"] = ["period1"]
# Save json template
with open(input_data_path / "Topology.json", "w") as json_file:
    json.dump(topology, json_file, indent=4)

In [5]:
# load json template
with open(input_data_path / "ConfigModel.json", "r") as json_file:
    configuration = json.load(json_file)
# Set time aggregation settings:
configuration["optimization"]["typicaldays"]["N"]["value"] = 30 
configuration["optimization"]["typicaldays"]["method"]["value"] = 1
# Set MILP gap
configuration["solveroptions"]["mipgap"]["value"] = 0.02
# save json template
with open(input_data_path / "ConfigModel.json", "w") as json_file:
    json.dump(configuration, json_file, indent=4)

In [6]:
adopt.create_input_data_folder_template(input_data_path)

# Define node locations (here two exemplary location in the Netherlands)
node_location = pd.read_csv(input_data_path / "NodeLocations.csv", sep=';', index_col=0, header=0)
node_lon = {'northwest': 8.5, 'northeast': 11.5,'center': 12.0, 'south': 15.5, 'islands': 14.0}  #longitude in degrees
node_lat = {'northwest': 45.5, 'northeast': 45.0, 'center': 42.5, 'south': 40.5, 'islands': 38.0}  # latitude in degrees
node_alt = {'northwest': 200, 'northeast': 100, 'center': 150, 'south': 300, 'islands': 400}    # Elevation in meters
for node in ['northwest', 'northeast', 'center', 'south', 'islands']:
    node_location.at[node, 'lon'] = node_lon[node]
    node_location.at[node, 'lat'] = node_lat[node]
    node_location.at[node, 'alt'] = node_alt[node]

node_location = node_location.reset_index()
node_location.to_csv(input_data_path / "NodeLocations.csv", sep=';', index=False)

adopt.show_available_technologies()

CO2Railway
CO2Ship
CO2Truck
CO2_Pipeline
electricityOffshore
electricityOnshore
electricitySimple
heat
hydrogenPipelineOffshore
hydrogenPipelineOnshore
hydrogenRailway
hydrogenShip
hydrogenSimple
hydrogenTruck
CO2_Compressor
MEA_large
MEA_medium
MEA_small
DAC_Adsorption
dac_adsorption_performanc
Boiler_El
Boiler_Industrial_NG
Boiler_Small_H2
Boiler_Small_NG
Furnace_H2
Furnace_NG
HeatPump_AirSourced
HeatPump_GroundSourced
HeatPump_SeaWaterSourced
HeatPump_WaterSourced
Hydro_Reservoir
PumpedHydro_Closed
PumpedHydro_Open
CrackerFurnace
CrackerFurnace_Electric
EthyleneCompression
EthyleneCompression_Electric
EthyleneSeparation
SteamReformer
SteamReformer_CCS
CombinedCycle_fixed_size
GasTurbine_H2_10
GasTurbine_H2_100
GasTurbine_H2_250
GasTurbine_H2_400
GasTurbine_NG_10
GasTurbine_NG_100
GasTurbine_NG_250
GasTurbine_NG_400
GasTurbine_simple
GasTurbine_simple_CCS
SteamTurbine
GT_fitting_dat
HP_fitting_dat
MP_fitting_dat
Photovoltaic
SolarThermal
WindTurbine_Offshore_11000
WindTurbine_Offshor

In [7]:
# Add required technologies for node 'North' (We will be changing this and also adding new technologies like CCS)      (Waste-to-energy)

#Northwest technological configuration
with open(input_data_path / "period1" / "node_data" / "northwest" / "Technologies.json", "r") as json_file:
    technologies = json.load(json_file)
technologies["new"] = ["HeatPump_AirSourced", "Storage_Battery", "Photovoltaic", "WindTurbine_Onshore_4000", "Storage_H2"]           
technologies["existing"] = {"Boiler_Small_NG": 3000, "GasTurbine_simple": 2500}    

with open(input_data_path / "period1" / "node_data" / "northwest" / "Technologies.json", "w") as json_file:
    json.dump(technologies, json_file, indent=4)


# NorthEast technological configuration
with open(input_data_path / "period1" / "node_data" / "northeast" / "Technologies.json", "r") as json_file:
    technologies = json.load(json_file)
technologies["new"] = ["HeatPump_AirSourced", "Storage_Battery", "Photovoltaic", "WindTurbine_Onshore_4000", "Storage_H2"]           
technologies["existing"] = {"Boiler_Small_NG": 2500, "GasTurbine_simple": 2000}      

with open(input_data_path / "period1" / "node_data" / "northeast" / "Technologies.json", "w") as json_file:
    json.dump(technologies, json_file, indent=4)


#Center technological configuration
with open(input_data_path / "period1" / "node_data" / "center" / "Technologies.json", "r") as json_file:
    technologies = json.load(json_file)
technologies["new"] = ["HeatPump_AirSourced", "Storage_Battery", "Photovoltaic", "WindTurbine_Onshore_4000", "Storage_H2", "Storage_H2"]           
technologies["existing"] = {"Boiler_Small_NG": 2800, "GasTurbine_simple": 1800}     

with open(input_data_path / "period1" / "node_data" / "center" / "Technologies.json", "w") as json_file:
    json.dump(technologies, json_file, indent=4)


# Add required technologies for node 'south'
with open(input_data_path / "period1" / "node_data" / "south" / "Technologies.json", "r") as json_file:
    technologies = json.load(json_file)
technologies["new"] = ["HeatPump_AirSourced", "Storage_Battery", "Photovoltaic", "WindTurbine_Onshore_4000", "WindTurbine_Offshore_9500", "Storage_H2"]  #WindTurbine_offshore  and also municipality boilers 
technologies["existing"] = {"Boiler_Small_NG": 2200, "GasTurbine_simple": 1500} #MWth and MWe

with open(input_data_path / "period1" / "node_data" / "south" / "Technologies.json", "w") as json_file:
    json.dump(technologies, json_file, indent=4)


#ISlands technological configuration
with open(input_data_path / "period1" / "node_data" / "islands" / "Technologies.json", "r") as json_file:
    technologies = json.load(json_file)
technologies["new"] = ["HeatPump_AirSourced", "Storage_Battery", "Photovoltaic", "WindTurbine_Onshore_4000", "WindTurbine_Offshore_9500", "Storage_H2"]  #WindTurbine_offshore  and also municipality boilers 
technologies["existing"] = {"Boiler_Small_NG": 1200, "GasTurbine_simple": 1000} #MWth and MWe

with open(input_data_path / "period1" / "node_data" / "islands" / "Technologies.json", "w") as json_file:
    json.dump(technologies, json_file, indent=4)


# Copy over technology files
adopt.copy_technology_data(input_data_path)

In [8]:
adopt.show_available_networks()

In [9]:
# Add networks (here we will add an onshore electricity network, liket the one in the case study as an example)
with open(input_data_path / "period1" / "Networks.json", "r") as json_file:
    networks = json.load(json_file)
networks["new"] = ["electricityOnshore"]
networks["existing"] = ["electricityOnshore"]

with open(input_data_path / "period1" / "Networks.json", "w") as json_file:
    json.dump(networks, json_file, indent=4)

In [10]:
# === Make a new folder for the existing network (We can also create the dataframe for this and import it.)
os.makedirs(input_data_path / "period1" / "network_topology" / "existing" / "electricityOnshore", exist_ok=True)

print("Existing network")

# === Connection (Existing)
connection = pd.read_csv(input_data_path / "period1" / "network_topology" / "existing" / "connection.csv", sep=";", index_col=0)
connection.loc["northwest", "northeast"] = 1
connection.loc["northeast", "northwest"] = 1
connection.loc["northwest", "center"] = 1
connection.loc["center", "northwest"] = 1
connection.loc["northeast", "center"] = 1
connection.loc["center", "northeast"] = 1
connection.loc["center", "south"] = 1
connection.loc["south", "center"] = 1
connection.loc["south", "islands"] = 1  # Sicily connection
connection.loc["islands", "south"] = 1
connection.to_csv(input_data_path / "period1" / "network_topology" / "existing" / "electricityOnshore" / "connection.csv", sep=";")
print("Connection:", connection)

# Delete the original template
os.remove(input_data_path / "period1" / "network_topology" / "existing" / "connection.csv")

# === Distance (Existing)
distance = pd.read_csv(input_data_path / "period1" / "network_topology" / "existing" / "distance.csv", sep=";", index_col=0)
distance.loc["northwest", "northeast"] = 400
distance.loc["northeast", "northwest"] = 400
distance.loc["northwest", "center"] = 500
distance.loc["center", "northwest"] = 500
distance.loc["northeast", "center"] = 450
distance.loc["center", "northeast"] = 450
distance.loc["center", "south"] = 350
distance.loc["south", "center"] = 350
distance.loc["south", "islands"] = 200
distance.loc["islands", "south"] = 200
distance.to_csv(input_data_path / "period1" / "network_topology" / "existing" / "electricityOnshore" / "distance.csv", sep=";")
print("Distance:", distance)

# Delete the original template
os.remove(input_data_path / "period1" / "network_topology" / "existing" / "distance.csv")

# === Size (Existing)
size = pd.read_csv(input_data_path / "period1" / "network_topology" / "existing" / "size.csv", sep=";", index_col=0)
size.loc["northwest", "northeast"] = 3000
size.loc["northeast", "northwest"] = 3000
size.loc["northwest", "center"] = 2500
size.loc["center", "northwest"] = 2500
size.loc["northeast", "center"] = 2800
size.loc["center", "northeast"] = 2800
size.loc["center", "south"] = 2200
size.loc["south", "center"] = 2200
size.loc["south", "islands"] = 1000
size.loc["islands", "south"] = 1000
size.to_csv(input_data_path / "period1" / "network_topology" / "existing" / "electricityOnshore" / "size.csv", sep=";")
print("Size:", size)

# Delete the original template
os.remove(input_data_path / "period1" / "network_topology" / "existing" / "size.csv")


print("New network")
# === Make a new folder for the new network
os.makedirs(input_data_path / "period1" / "network_topology" / "new" / "electricityOnshore", exist_ok=True)

# === Max Size Arc (New)
arc_size = pd.read_csv(input_data_path / "period1" / "network_topology" / "new" / "size_max_arcs.csv", sep=";", index_col=0)
arc_size.loc["northwest", "northeast"] = 5000
arc_size.loc["northeast", "northwest"] = 5000
arc_size.loc["northwest", "center"] = 4000
arc_size.loc["center", "northwest"] = 4000
arc_size.loc["northeast", "center"] = 4500
arc_size.loc["center", "northeast"] = 4500
arc_size.loc["center", "south"] = 4000
arc_size.loc["south", "center"] = 4000
arc_size.loc["south", "islands"] = 2000
arc_size.loc["islands", "south"] = 2000
arc_size.loc["northwest", "south"] = 6000
arc_size.loc["south", "northwest"] = 6000
arc_size.loc["northwest", "islands"] = 7000
arc_size.loc["islands", "northwest"] = 7000
arc_size.loc["northeast", "south"] = 6500
arc_size.loc["south", "northeast"] = 6500
arc_size.loc["northeast", "islands"] = 7500
arc_size.loc["islands", "northeast"] = 7500
arc_size.loc["center", "islands"] = 5000
arc_size.loc["islands", "center"] = 5000
arc_size.to_csv(input_data_path / "period1" / "network_topology" / "new" / "electricityOnshore" / "size_max_arcs.csv", sep=";")
print("Max size per arc:", arc_size)

# === Connection (New)
connection = pd.read_csv(input_data_path / "period1" / "network_topology" / "new" / "connection.csv", sep=";", index_col=0)
connection.loc["northwest", "northeast"] = 1
connection.loc["northeast", "northwest"] = 1
connection.loc["northwest", "center"] = 1
connection.loc["center", "northwest"] = 1
connection.loc["northeast", "center"] = 1
connection.loc["center", "northeast"] = 1
connection.loc["center", "south"] = 1
connection.loc["south", "center"] = 1
connection.loc["south", "islands"] = 1
connection.loc["islands", "south"] = 1
connection.loc["northwest", "south"] = 1
connection.loc["south", "northwest"] = 1
connection.loc["northwest", "islands"] = 1
connection.loc["islands", "northwest"] = 1
connection.loc["northeast", "south"] = 1
connection.loc["south", "northeast"] = 1
connection.loc["northeast", "islands"] = 1
connection.loc["islands", "northeast"] = 1
connection.loc["center", "islands"] = 1
connection.loc["islands", "center"] = 1
connection.to_csv(input_data_path / "period1" / "network_topology" / "new" / "electricityOnshore" / "connection.csv", sep=";")
print("Connection:", connection)

# Delete connection template
os.remove(input_data_path / "period1" / "network_topology" / "new" / "connection.csv")

# === Distance (New)
distance = pd.read_csv(input_data_path / "period1" / "network_topology" / "new" / "distance.csv", sep=";", index_col=0)
distance.loc["northwest", "northeast"] = 400   # Turin/Milan to Venice/Trieste
distance.loc["northeast", "northwest"] = 400
distance.loc["northwest", "center"] = 500      # Turin/Milan to Rome
distance.loc["center", "northwest"] = 500
distance.loc["northwest", "south"] = 900       # Turin/Milan to Naples/Bari
distance.loc["south", "northwest"] = 900
distance.loc["northwest", "islands"] = 1100    # Turin/Milan to Palermo/Cagliari
distance.loc["islands", "northwest"] = 1100
distance.loc["northeast", "center"] = 450      # Venice to Rome
distance.loc["center", "northeast"] = 450
distance.loc["northeast", "south"] = 750       # Venice to Naples/Bari
distance.loc["south", "northeast"] = 750
distance.loc["northeast", "islands"] = 1000    # Venice to Palermo/Cagliari
distance.loc["islands", "northeast"] = 1000
distance.loc["center", "south"] = 350          # Rome to Naples
distance.loc["south", "center"] = 350
distance.loc["center", "islands"] = 600        # Rome to Palermo/Cagliari
distance.loc["islands", "center"] = 600
distance.loc["south", "islands"] = 200         # Naples/Bari to Palermo/Cagliari
distance.loc["islands", "south"] = 200
distance.to_csv(input_data_path / "period1" / "network_topology" / "new" / "electricityOnshore" / "distance.csv", sep=";")
print("Distance:", distance)

# Delete distance template
os.remove(input_data_path / "period1" / "network_topology" / "new" / "distance.csv")

# Delete size_max_arcs template
os.remove(input_data_path / "period1" / "network_topology" / "new" / "size_max_arcs.csv")

Existing network
Connection:            northwest  northeast  center  south  islands
northwest          0          1       1      0        0
northeast          1          0       1      0        0
center             1          1       0      1        0
south              0          0       1      0        1
islands            0          0       0      1        0
Distance:            northwest  northeast  center  south  islands
northwest          0        400     500      0        0
northeast        400          0     450      0        0
center           500        450       0    350        0
south              0          0     350      0      200
islands            0          0       0    200        0
Size:            northwest  northeast  center  south  islands
northwest          0       3000    2500      0        0
northeast       3000          0    2800      0        0
center          2500       2800       0   2200        0
south              0          0    2200      0     1000
isl

In [11]:
adopt.copy_network_data(input_data_path)

with open(input_data_path / "period1" / "network_data"/ "electricityOnshore.json", "r") as json_file:
    network_data = json.load(json_file)

network_data["Economics"]["gamma2"] = 50000
network_data["Economics"]["gamma4"] = 400

with open(input_data_path / "period1" / "network_data"/ "electricityOnshore.json", "w") as json_file:
    json.dump(network_data, json_file, indent=4)

In [12]:
#Demand, climate data, import prices and limits 
# Read hourly data from Excel (example demand profiles) Chnage this to your own data of demand 
islands_hourly_data = pd.read_excel(input_data_path / "data/regional_demand_profiles.xlsx", sheet_name="islands", nrows=8760)
south_hourly_data = pd.read_excel(input_data_path / "data/regional_demand_profiles.xlsx", sheet_name="south", nrows=8760)
center_hourly_data = pd.read_excel(input_data_path / "data/regional_demand_profiles.xlsx", sheet_name="center", nrows=8760)
northeast_hourly_data = pd.read_excel(input_data_path  / "data/regional_demand_profiles.xlsx", sheet_name="northeast", nrows=8760)
northwest_hourly_data = pd.read_excel(input_data_path / "data/regional_demand_profiles.xlsx", sheet_name="northwest", nrows=8760)

# Save the hourly data to the carrier's file in the case study folder
hourly_data = {
    'northwest': northwest_hourly_data,
    'northeast': northeast_hourly_data,
    'center': center_hourly_data,
    'south': south_hourly_data,
    'islands': islands_hourly_data
}

el_demand = {}
heat_demand = {}

for node in ['northwest', 'northeast', 'center', 'south', 'islands']:
    el_demand[node] = hourly_data[node].iloc[:, 0]  # Electricity demand
    heat_demand[node] = hourly_data[node].iloc[:, 1]  # Heat demand

    adopt.fill_carrier_data(input_data_path, value_or_data=el_demand[node], columns=['Demand'], carriers=['electricity'], nodes=[node])
    adopt.fill_carrier_data(input_data_path, value_or_data=heat_demand[node], columns=['Demand'], carriers=['heat'], nodes=[node])

    #Set import limits and costs
    adopt.fill_carrier_data(input_data_path, value_or_data=8000, columns=['Import limit'], carriers=['gas'], nodes=[node])
    adopt.fill_carrier_data(input_data_path, value_or_data=3000, columns=['Import limit'], carriers=['electricity'], nodes=[node])
    adopt.fill_carrier_data(input_data_path, value_or_data=0.35, columns=['Import emission factor'], carriers=['electricity'], nodes=[node])
    adopt.fill_carrier_data(input_data_path, value_or_data=45, columns=['Import price'], carriers=['gas'], nodes=[node])
    adopt.fill_carrier_data(input_data_path, value_or_data=150, columns=['Import price'], carriers=['electricity'], nodes=[node])

#Define climate data
adopt.load_climate_data_from_api(input_data_path)


Importing Climate Data...
Importing Climate Data successful
Importing Climate Data...
Importing Climate Data successful
Importing Climate Data...
Importing Climate Data successful
Importing Climate Data...
Importing Climate Data successful
Importing Climate Data...
Importing Climate Data successful


In [13]:
# Run the optimization model with the baseline scenario (no ETS)
m = adopt.ModelHub()
m.read_data(input_data_path)
m.quick_solve()

--- Reading in data ---
Input data folder has been checked successfully - no errors occurred.
Reading data from macro_decarbonisation
Topology read successfully
Model Configuration read successfully
Time series read successfully
Node Locations read successfully
Energy balance options read successfully
Deriving performance data for Heat Pump...


Complete:  99.0 %Complete:  100 %


Deriving performance data for Heat Pump...


Complete:  99.0 %Complete:  100 %


Deriving performance data for Heat Pump...


...

Best objective 4.712235940852e+09, best bound 4.712235940852e+09, gap 0.0000%


Set parameter LogFile to value ""


Set parameter LogFile to value ""
Writing results to userData\20250829132717-1


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 4712235940.851585
  Upper bound: 4712235940.851585
  Number of objectives: 1
  Number of constraints: 734326
  Number of variables: 571083
  Number of binary variables: 36040
  Number of integer variables: 41087
  Number of continuous variables: 493956
  Number of nonzeros: 1730414
  Sense: minimize
  Number of solutions: 1
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Name: Gurobi 12.02
  Status: ok
  Wallclock time: 19.301000118255615
  Termination condition: optimal
  Termination message: Model was solved to opt

Solving model completed in 75 s


The polices of the ETS1 (POWER SECTOR CARBON PRICING)

In [14]:
print("Implementing ETS1 carbon pricing for power sector...")

# ETS1 carbon price: ~85 EUR/tCO2 (2021 levels)
carbon_price = np.ones(8760) * 85

# Apply ETS1 to power sector (affects electricity generation and imports)
for node in ['northwest', 'northeast', 'center', 'south', 'islands']:
    carbon_cost_path = "./macro_decarbonisation/period1/node_data/" + node + "/CarbonCost.csv"
    carbon_cost_template = pd.read_csv(carbon_cost_path, sep=';', index_col=0, header=0)
    carbon_cost_template['price'] = carbon_price
    carbon_cost_template = carbon_cost_template.reset_index()
    carbon_cost_template.to_csv(carbon_cost_path, sep=';', index=False)

# Run the optimization model with ETS1 scenario
m = adopt.ModelHub()
m.read_data(input_data_path)
m.quick_solve()

--- Reading in data ---
Input data folder has been checked successfully - no errors occurred.
Reading data from macro_decarbonisation
Topology read successfully
Model Configuration read successfully


Implementing ETS1 carbon pricing for power sector...


Time series read successfully
Node Locations read successfully
Energy balance options read successfully
Deriving performance data for Heat Pump...


Complete:  99.0 %Complete:  100 %


Deriving performance data for Heat Pump...


...

Best objective 7.518821308290e+09, best bound 7.518821308290e+09, gap 0.0000%


Set parameter LogFile to value ""


Set parameter LogFile to value ""
Writing results to userData\20250829133457-1


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 7518821308.290333
  Upper bound: 7518821308.290348
  Number of objectives: 1
  Number of constraints: 734326
  Number of variables: 571083
  Number of binary variables: 36040
  Number of integer variables: 41087
  Number of continuous variables: 493956
  Number of nonzeros: 1767866
  Sense: minimize
  Number of solutions: 1
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Name: Gurobi 12.02
  Status: ok
  Wallclock time: 28.983999967575073
  Termination condition: optimal
  Termination message: Model was solved to opt

Solving model completed in 92 s


RUN OPTIMIZATION WITH ETS1 + ETS2 (COMPREHENSIVE CARBON PRICING)
print("Implementing ETS1 + ETS2 carbon pricing")


In [15]:
# ETS2 carbon price: ~50 EUR/tCO2 (projected for buildings/heating)
# Combined effect: higher carbon pricing across all sectors
carbon_price = np.ones(8760) * 120  # Higher combined effect

# Apply combined ETS1+ETS2 carbon pricing
for node in ['northwest', 'northeast', 'center', 'south', 'islands']:
    carbon_cost_path = "./macro_decarbonisation/period1/node_data/" + node + "/CarbonCost.csv"
    carbon_cost_template = pd.read_csv(carbon_cost_path, sep=';', index_col=0, header=0)
    carbon_cost_template['price'] = carbon_price
    carbon_cost_template = carbon_cost_template.reset_index()
    carbon_cost_template.to_csv(carbon_cost_path, sep=';', index=False)

# Run the optimization model with ETS1+ETS2 scenario
m = adopt.ModelHub()
m.read_data(input_data_path)
m.quick_solve()


--- Reading in data ---
Input data folder has been checked successfully - no errors occurred.
Reading data from macro_decarbonisation
Topology read successfully
Model Configuration read successfully
Time series read successfully
Node Locations read successfully
Energy balance options read successfully
Deriving performance data for Heat Pump...


Complete:  99.0 %Complete:  100 %


Deriving performance data for Heat Pump...


Complete:  99.0 %Complete:  100 %


Deriving performance data for Heat Pump...


...

Best objective 8.047398408353e+09, best bound 8.047398408353e+09, gap 0.0000%


Set parameter LogFile to value ""


Set parameter LogFile to value ""
Writing results to userData\20250829134246-1


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 8047398408.353289
  Upper bound: 8047398408.353291
  Number of objectives: 1
  Number of constraints: 734326
  Number of variables: 571083
  Number of binary variables: 36040
  Number of integer variables: 41087
  Number of continuous variables: 493956
  Number of nonzeros: 1767857
  Sense: minimize
  Number of solutions: 1
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Name: Gurobi 12.02
  Status: ok
  Wallclock time: 25.39800000190735
  Termination condition: optimal
  Termination message: Model was solved to opti

Solving model completed in 96 s
